In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet.diagnostics import cross_validation
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 
set_random_seed(0)

In [2]:
data = pd.read_csv('../Data/all_in_one.csv')
data.head()

,Datetime,temp,desc,icon,baro,wind,wd,hum,rain,fog,...,curfew,etat_barre_ce,q_ce,k_ce,etat_barre_lc,q_lc,k_lc,etat_barre_pv,q_pv,k_pv
0,2014-01-01 00:00:00,7.0,Low clouds.,17.0,1011.0,15.0,220.0,92.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01 01:00:00,6.0,Low clouds.,17.0,1011.0,15.0,220.0,92.0,0.0,0.0,...,0.0,3.0,4.0,0.00000,3.0,205.0,2.25945,3.0,486.0,6.17889
2,2014-01-01 02:00:00,6.0,Low clouds.,17.0,1011.0,15.0,220.0,92.0,0.0,0.0,...,0.0,3.0,21.0,0.10445,3.0,283.0,3.25111,3.0,571.0,6.66222
3,2014-01-01 03:00:00,6.0,Low clouds.,17.0,1011.0,15.0,220.0,92.0,0.0,0.0,...,0.0,3.0,417.0,6.12667,3.0,309.0,3.36000,3.0,532.0,5.56611
4,2014-01-01 04:00:00,6.0,Low clouds.,17.0,1011.0,15.0,220.0,92.0,0.0,0.0,...,0.0,3.0,1089.0,14.39890,3.0,277.0,3.01445,3.0,458.0,4.63444


In [3]:
data.describe()

,temp,icon,baro,wind,wd,hum,rain,fog,thunder,snow,...,curfew,etat_barre_ce,q_ce,k_ce,etat_barre_lc,q_lc,k_lc,etat_barre_pv,q_pv,k_pv
count,60505.000000,60648.000000,60648.000000,60648.000000,60648.000000,60648.000000,61368.000000,61368.000000,61368.000000,61368.000000,...,61368.000000,60545.000000,56554.000000,55899.000000,60548.000000,59582.000000,59668.000000,60547.000000,58906.000000,58907.000000
mean,12.380960,9.612881,1014.118619,12.511674,171.266324,74.772556,0.051525,0.080172,0.002346,0.001564,...,0.005149,2.983880,1179.864324,9.751981,2.999835,499.960642,6.632829,2.969990,483.401063,7.433277
std,6.983427,6.946060,50.676184,6.074135,110.274874,17.679752,0.221068,0.271562,0.048384,0.039521,...,0.071574,0.125939,625.661223,7.915059,0.012850,309.902459,8.255011,0.170615,305.480100,6.527415
min,-8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.000000,0.000000,0.000000,2.000000,8.000000,0.016670,2.000000,0.000000,0.000000
25%,7.000000,2.000000,1012.000000,8.000000,60.000000,63.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,668.000000,4.815560,3.000000,222.000000,1.658330,3.000000,237.000000,2.761945
50%,12.000000,9.000000,1017.000000,12.000000,200.000000,79.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,1144.000000,8.486670,3.000000,480.000000,3.790280,3.000000,503.000000,6.034440
75%,17.000000,14.000000,1022.000000,16.000000,250.000000,89.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,1708.000000,11.392200,3.000000,751.000000,7.691110,3.000000,701.000000,9.812780
max,40.000000,36.000000,1046.000000,45.000000,350.000000,100.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,3.000000,6530.000000,84.668890,3.000000,1386.000000,65.470000,3.000000,33272.000000,100.000000


In [4]:
data = data[(data.Datetime < '2020-11-30') & (data.Datetime >= '2018-01-01')]

In [5]:
data.describe()

,temp,icon,baro,wind,wd,hum,rain,fog,thunder,snow,...,curfew,etat_barre_ce,q_ce,k_ce,etat_barre_lc,q_lc,k_lc,etat_barre_pv,q_pv,k_pv
count,25206.000000,25296.000000,25296.000000,25296.000000,25296.000000,25296.000000,25536.000000,25536.000000,25536.000000,25536.000000,...,25536.000000,25483.000000,22851.000000,22278.000000,25486.000000,25249.000000,25256.000000,25485.000000,24152.000000,24153.000000
mean,12.842260,9.662951,1012.470114,13.205408,164.592030,73.117173,0.050282,0.075658,0.002820,0.003289,...,0.005091,2.978378,884.838563,10.808077,2.999882,588.272050,7.034260,2.929880,406.892224,5.756469
std,7.238643,7.097960,61.207146,6.300860,111.144594,18.657378,0.218530,0.264455,0.053026,0.057261,...,0.071170,0.145450,492.173746,9.708492,0.010849,336.412841,8.926217,0.255354,333.415803,4.939353
min,-8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.000000,0.000000,0.000000,2.000000,8.000000,0.016670,2.000000,1.000000,0.000000
25%,7.500000,2.000000,1011.000000,9.000000,50.000000,59.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,469.000000,4.150278,3.000000,273.000000,1.576532,3.000000,161.000000,1.928330
50%,12.000000,9.000000,1017.000000,12.000000,190.000000,77.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,925.000000,8.038055,3.000000,596.000000,3.614170,3.000000,420.000000,5.008330
75%,18.000000,14.000000,1022.000000,17.000000,250.000000,88.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.000000,1190.000000,14.166528,3.000000,891.000000,8.037637,3.000000,613.000000,7.989450
max,40.000000,36.000000,1046.000000,45.000000,350.000000,100.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,3.000000,2359.000000,84.668890,3.000000,1386.000000,65.470000,3.000000,33272.000000,100.000000


# Baseline

## Data

## Champs Elysées

### Prophet normal

In [6]:
X_q = data[['Datetime', 'q_ce']]
X_k = data[['Datetime', 'k_ce']]
X_q.columns = ['ds', 'y']
X_k.columns = ['ds', 'y']

In [31]:
model_q = Prophet()
model_k = Prophet()
model_q.fit(X_q)
model_k.fit(X_k)

In [36]:
df_cv_q = cross_validation(model_q, initial='1022 days', period='5 days', horizon = '5 days')
df_cv_k = cross_validation(model_k, initial='1022 days', period='5 days', horizon = '5 days')

INFO:fbprophet:Making 8 forecasts with cutoffs between 2020-10-20 23:00:00 and 2020-11-24 23:00:00


INFO:fbprophet:Making 8 forecasts with cutoffs between 2020-10-20 23:00:00 and 2020-11-24 23:00:00


In [40]:
df_p_q = performance_metrics(df_cv_q)
df_p_k = performance_metrics(df_cv_k)

In [46]:
print(f"RMSE for q = {df_p_q.rmse.mean()}")
print(f"RMSE for k = {df_p_k.rmse.mean()}")

RMSE for q = 306.55536225771084
RMSE for k = 5.31593423362365


### Neural Prophet

In [7]:
## FILL MISSING VALUES AS NEURALPROPHET CAN'T HANDLE THEM
X_q = pd.DataFrame({'y': X_q.y.tolist()}, index=pd.to_datetime(X_q.ds)).interpolate(method='time').reset_index()
X_k = pd.DataFrame({'y': X_k.y.tolist()}, index=pd.to_datetime(X_k.ds)).interpolate(method='time').reset_index()

In [8]:
forecast = ['2020-10-21', '2020-10-26', '2020-10-31', '2020-11-05', '2020-11-10', '2020-11-15', '2020-11-20', '2020-11-25']
my_x_q = [X_q[X_q.ds < x] for x in forecast]
my_x_k = [X_k[X_k.ds < x] for x in forecast]

In [9]:
lst_model_q = []
lst_model_k = []
for data_q in my_x_q:
    model_q = NeuralProphet()
    metrics = model_q.fit(data_q, freq='h', epochs=60)
    lst_model_q.append(model_q)
for data_k in my_x_k:
    model_k = NeuralProphet()
    metrics = model_k.fit(data_k, freq='h', epochs=60)
    lst_model_k.append(model_k)

INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 2.31E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:38<00:00,  1.55it/s, SmoothL1Loss=0.0144, MAE=244, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 2.31E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:38<00:00,  1.55it/s, SmoothL1Loss=0.0145, MAE=244, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 2.31E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:42<00:00,  1.40it/s, SmoothL1Loss=0.0146, MAE=245, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 1.87E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:40<00:00,  1.47it/s, SmoothL1Loss=0.0147, MAE=246, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


...

INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 2.85E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:40<00:00,  1.47it/s, SmoothL1Loss=0.024, MAE=4.44, RegLoss=0] 
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 3.51E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:40<00:00,  1.48it/s, SmoothL1Loss=0.0241, MAE=4.45, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 5.34E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:40<00:00,  1.47it/s, SmoothL1Loss=0.0241, MAE=4.47, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 8.11E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:40<00:00,  1.47it/s, SmoothL1Loss=0.0241, MAE=4.47, RegLoss=0]
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 128


INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 2.31E-01
Epoch[60/60]: 100%|██████████| 60/60 [00:39<00:00,  1.50it/s, SmoothL1Loss=0.0241, MAE=4.43, RegLoss=0]


In [35]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [36]:
forecast = ['2020-10-21', '2020-10-26', '2020-10-31', '2020-11-05', '2020-11-10', '2020-11-15', '2020-11-20', '2020-11-25', '2020-11-30']
rmse_q = []
rmse_k = []
for index, x in enumerate(zip(lst_model_q, my_x_q)):
    model, df, i = x[0], x[1], index
    future = model.make_future_dataframe(df, periods=120)
    y_pred, y_true = model.predict(future).yhat1.values, X_q[(X_q.ds >= forecast[i]) & (X_q.ds < forecast[i+1])].y.values
    rmse_q.append(rmse(y_true, y_pred))
for index, x in enumerate(zip(lst_model_k, my_x_k)):
    model, df, i = x[0], x[1], index
    future = model.make_future_dataframe(df, periods=120)
    y_pred, y_true = model.predict(future).yhat1.values, X_k[(X_k.ds >= forecast[i]) & (X_k.ds < forecast[i+1])].y.values
    rmse_k.append(rmse(y_true, y_pred))

In [38]:
print(f"RMSE for q = {sum(rmse_q)/len(rmse_q)}")
print(f"RMSE for k = {sum(rmse_k)/len(rmse_k)}")

RMSE for q = 324.8281161055366
RMSE for k = 5.715858301351767


# Baseline with less data

In [39]:
data = data[(data.Datetime < '2020-11-30') & (data.Datetime >= '2019-10-01')]
X_q = data[['Datetime', 'q_ce']]
X_k = data[['Datetime', 'k_ce']]
X_q.columns = ['ds', 'y']
X_k.columns = ['ds', 'y']
## FILL MISSING VALUES AS NEURALPROPHET CAN'T HANDLE THEM
X_q = pd.DataFrame({'y': X_q.y.tolist()}, index=pd.to_datetime(X_q.ds)).interpolate(method='time').reset_index()
X_k = pd.DataFrame({'y': X_k.y.tolist()}, index=pd.to_datetime(X_k.ds)).interpolate(method='time').reset_index()
forecast = ['2020-10-21', '2020-10-26', '2020-10-31', '2020-11-05', '2020-11-10', '2020-11-15', '2020-11-20', '2020-11-25']
my_x_q = [X_q[X_q.ds < x] for x in forecast]
my_x_k = [X_k[X_k.ds < x] for x in forecast]

In [47]:
lst_model_q = []
lst_model_k = []
for data_q in my_x_q:
    model_q = NeuralProphet(learning_rate = 0.001)
    metrics = model_q.fit(data_q, freq='h', epochs=100)
    lst_model_q.append(model_q)
for data_k in my_x_k:
    model_k = NeuralProphet( learning_rate = 0.001)
    metrics = model_k.fit(data_k, freq='h', epochs=100)
    lst_model_k.append(model_k)

INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 32
Epoch[100/100]: 100%|██████████| 100/100 [00:42<00:00,  2.38it/s, SmoothL1Loss=0.0142, MAE=197, RegLoss=0] 
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 32
Epoch[100/100]: 100%|██████████| 100/100 [00:42<00:00,  2.34it/s, SmoothL1Loss=0.0145, MAE=200, RegLoss=0] 
INFO: nprophet.utils - set_auto_seasonalities: Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 32
Epoch[100/100]: 100%|██████████| 100/100 [00:43<00:00,  2.31it/s, SmoothL1Loss=0.0151, MAE=204, RegLoss=0]
INFO: nprophet.utils - set_auto_sea

In [48]:
forecast = ['2020-10-21', '2020-10-26', '2020-10-31', '2020-11-05', '2020-11-10', '2020-11-15', '2020-11-20', '2020-11-25', '2020-11-30']
rmse_q = []
rmse_k = []
for index, x in enumerate(zip(lst_model_q, my_x_q)):
    model, df, i = x[0], x[1], index
    future = model.make_future_dataframe(df, periods=120)
    y_pred, y_true = model.predict(future).yhat1.values, X_q[(X_q.ds >= forecast[i]) & (X_q.ds < forecast[i+1])].y.values
    rmse_q.append(rmse(y_true, y_pred))
for index, x in enumerate(zip(lst_model_k, my_x_k)):
    model, df, i = x[0], x[1], index
    future = model.make_future_dataframe(df, periods=120)
    y_pred, y_true = model.predict(future).yhat1.values, X_k[(X_k.ds >= forecast[i]) & (X_k.ds < forecast[i+1])].y.values
    rmse_k.append(rmse(y_true, y_pred))

In [49]:
print(f"RMSE for q = {sum(rmse_q)/len(rmse_q)}")
print(f"RMSE for k = {sum(rmse_k)/len(rmse_k)}")

RMSE for q = 327.7938334312713
RMSE for k = 6.443125430269929


The errors increased ..